<a href="https://colab.research.google.com/github/sellerstx1982/multimodal_damage_identification/blob/Irfan/project_3_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pathlib import Path
from sklearn.metrics import classification_report, balanced_accuracy_score, accuracy_score, classification_report
from PIL import Image, ImageFile
import os
import matplotlib.pyplot as plt
import zipfile

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pickle
#Load the data from pkl files
def load_data_from_pkl(pkl_filename):
    with open(pkl_filename, 'rb') as f:
        sample_df = pickle.load(f)
    return sample_df

df = load_data_from_pkl('/content/drive/MyDrive/disaster_data.pkl')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4436 to 4841
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1000 non-null   object
 1   text     1000 non-null   object
 2   label    1000 non-null   object
 3   label_2  1000 non-null   object
 4   label_3  1000 non-null   object
dtypes: object(5)
memory usage: 46.9+ KB


# Preprocessing

In [ ]:
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# main_folder = "/content/drive/MyDrive/multimodal+damage+identification+for+humanitarian+computing/multimodal/"

In [ ]:
# # Lists to store the data
# images = []
# texts = []
# labels = []

# # Preprocessing function for images
# def preprocess_image(image_path):
#     try:
#         image = Image.open(image_path)
#         image = image.resize((250, 250))  # Example size, adjust accordingly
#         image = np.array(image) / 255.0  # Normalize pixel values
#         return np.expand_dims(image, axis=0)  # Add batch dimension
#     except Exception as e:
#         print(f"Error processing image {image_path}: {e}")
#         return None

# # Function to load text from a file
# def load_text_file(text_path):
#     try:
#         with open(text_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     except Exception as e:
#         print(f"Error reading text file {text_path}: {e}")
#         return None

In [ ]:
# # Iterate over each category subfolder in the main folder
# for subfolder in os.listdir(main_folder):
#     subfolder_path = os.path.join(main_folder, subfolder)
#     if os.path.isdir(subfolder_path):
#         # The image and text folders inside each category folder
#         image_folder = os.path.join(subfolder_path, 'images')
#         text_folder = os.path.join(subfolder_path, 'text')

#         # Check if both the image and text folders exist
#         if os.path.exists(image_folder) and os.path.exists(text_folder):
#             for image_file in os.listdir(image_folder):
#                 if image_file.endswith(('jpg', 'jpeg', 'png')):
#                     image_path = os.path.join(image_folder, image_file)
#                     # Construct the corresponding text file name
#                     text_file = image_file.rsplit('.', 1)[0] + '.txt'
#                     text_path = os.path.join(text_folder, text_file)

#                     # Check if the text file exists and load both image and text
#                     if os.path.exists(text_path):
#                         image = preprocess_image(image_path)
#                         if image is not None:
#                             text = load_text_file(text_path)
#                             if text is not None:
#                                 # Extract the label by splitting at the underscore and taking the first part
#                                 label = image_file.split('_', 1)[0]
#                                 images.append(image)
#                                 texts.append(text)
#                                 labels.append(label)  # Append the prefix before the first underscore as label


In [ ]:
# # images = np.array([np.array(image) for image in images])
# type(images)

list

In [ ]:
# print(len(images))
# print(len(texts))
# print(len(labels))

5831
5831
5831


In [ ]:
# df = pd.DataFrame({'image': images, 'text': texts, 'label': labels})
# df.head()

,image,text,label
0,"[[[[0.82352941 0.90588235 0.97254902], [0.8235...",#isiscrimes \nIsis dogs crucified a men accuse...,isiscrimes
1,"[[[[0.99607843 0.99607843 0.99607843], [0.9960...",Part 2:\n\nIn January of this year Abu Rayyan ...,isiscrimes
2,"[[[[0.99607843 0.99607843 0.99607843], [0.9960...",Wake up.ppl see the reality to what ISIS is al...,isiscrimes
3,"[[[[0.58823529 0.44705882 0.34509804], [0.5803...",New image from akhtarin in aleppo #Syria yeste...,isiscrimes
4,"[[[[0.4627451 0.38039216 0.2745098 ], [0.4627...",Syria #Aleppo ISIS continue on with their behe...,isiscrimes


In [ ]:
# df['label_2'] = df['label'].replace({'destruction': 'destruction',
#                                    'drought': 'drought',
#                                    'buildingcollapse': 'earthquake',
#                                    'destroyedbuilding': 'earthquake',
#                                    'earthquake': 'earthquake',
#                                    'earthquakenepal': 'earthquake',
#                                    'buildingfire': 'fire',
#                                    'forestfire': 'fire',
#                                    'forrestfire': 'fire',
#                                    'wildfires': 'fire',
#                                    'accrafloods': 'flood',
#                                    'floodwater': 'flood',
#                                    'hurricaneharvey': 'hurricane',
#                                    'hurricaneirma': 'hurricane',
#                                    'hurricanematthew': 'hurricane',
#                                    'hurricanesandy': 'hurricane',
#                                    'sandydamage': 'hurricane',
#                                    'treefalling': 'hurricane',
#                                    'treesfalling': 'hurricane',
#                                    'windstorm': 'hurricane',
#                                    'landslide': 'landslide',
#                                    'disaster': 'naturaldisaster',
#                                    'disasters': 'naturaldisaster',
#                                    'naturaldamage': 'naturaldisaster',
#                                    'naturaldisaster': 'naturaldisaster',
#                                    'ad': 'non-damage',
#                                    'building': 'non-damage',
#                                    'cars': 'non-damage',
#                                    'food': 'non-damage',
#                                    'nature': 'non-damage',
#                                    'terrorattack': 'war',
#                                    'isiscrimes': 'war',
#                                    'yemencrisis': 'war',
#                                    'suicidebombing': 'war',
#                                    'victimsofwar': 'war',
#                                    'war': 'war',
#                                    'syriawarcrimes': 'war',
#                                    'warsyria': 'war',
#                                    'yemencrisis': 'war',
#                                    'wreckedcar': 'wreckedcar',
#                                    })

In [ ]:
# df['label_3'] = df['label'].replace({'destruction': 'damage',
#                                    'drought': 'damage',
#                                    'buildingcollapse': 'damage',
#                                    'destroyedbuilding': 'damage',
#                                    'earthquake': 'damage',
#                                    'earthquakenepal': 'damage',
#                                    'buildingfire': 'damage',
#                                    'forestfire': 'damage',
#                                    'forrestfire': 'damage',
#                                    'wildfires': 'damage',
#                                    'accrafloods': 'damage',
#                                    'floodwater': 'damage',
#                                    'hurricaneharvey': 'damage',
#                                    'hurricaneirma': 'damage',
#                                    'hurricanematthew': 'damage',
#                                    'hurricanesandy': 'damage',
#                                    'sandydamage': 'damage',
#                                    'treefalling': 'damage',
#                                    'treesfalling': 'damage',
#                                    'windstorm': 'damage',
#                                    'landslide': 'damage',
#                                    'disaster': 'damage',
#                                    'disasters': 'damage',
#                                    'naturaldamage': 'damage',
#                                    'naturaldisaster': 'damage',
#                                    'ad': 'non-damage',
#                                    'building': 'non-damage',
#                                    'cars': 'non-damage',
#                                    'food': 'non-damage',
#                                    'nature': 'non-damage',
#                                    'terrorattack': 'damage',
#                                    'isiscrimes': 'damage',
#                                    'yemencrisis': 'damage',
#                                    'suicidebombing': 'damage',
#                                    'victimsofwar': 'damage',
#                                    'war': 'damage',
#                                    'syriawarcrimes': 'damage',
#                                    'warsyria': 'damage',
#                                    'yemencrisis': 'damage',
#                                    'wreckedcar': 'damage',

#                                    })

In [6]:
df.head()

,image,text,label,label_2,label_3
4436,"[[[[0.48235294 0.6627451 0.90588235], [0.4823...",It's always the moment to install solar contro...,building,non-damage,non-damage
5297,"[[[[1. 1. 1.], [1. 1. 1.], [1. 1. 1.], [1. 1. ...",Esmond lake \n@motobudz91 \n#lake #log #oregon...,landslide,landslide,damage
1121,"[[[[0.6745098 0.70980392 0.83529412], [0.6745...",Goodbye suit corner. #buildingcollapse #accide...,buildingcollapse,earthquake,damage
561,"[[[[0.88627451 0.8745098 0.83921569], [0.5725...",#somalia is a country that the world does not ...,terrorattack,war,damage
1188,"[[[[0.71372549 0.70196078 0.76862745], [0.7137...",NJ State Troopers patrolling The Streets of Un...,sandydamage,hurricane,damage


In [ ]:
df['label_3'].value_counts()

,count
label_3,
non-damage,513
damage,487


In [ ]:
# sample_df = df.sample(n=1000, random_state=42)
# sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4436 to 4841
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1000 non-null   object
 1   text     1000 non-null   object
 2   label    1000 non-null   object
 3   label_2  1000 non-null   object
 4   label_3  1000 non-null   object
dtypes: object(5)
memory usage: 46.9+ KB


In [ ]:
# sample_df['label_3'].value_counts()

,count
label_3,
non-damage,513
damage,487


In [ ]:
df.head()

,image,text,label,label_2,label_3
4436,"[[[[0.14509804 0.12941176 0.03137255], [0.1450...",#vulcan#vulcaninsel#instagram#instavulcan#volc...,earthquake,earthquake,damage
5297,"[[[[0.91372549 0.91372549 0.96078431], [0.9137...",Making the best of the flood waters this morni...,floodwater,flood,damage
1121,"[[[[0.98431373 0.98431373 0.98431373], [0.9843...",*كلام حلو* .\n.\n\n#streetworkout #streetstyle...,ad,non-damage,non-damage
561,"[[[[0.97647059 0.97647059 0.97647059], [0.9568...",do you like this😜😜\n#\n.\nvia @freschle\n.\n.\...,treefalling,hurricane,damage
1188,"[[[[0.04705882 0.0627451 0.10588235], [0.0392...",WhatsApp On +919466780004 #earrings #jhumki #c...,ad,non-damage,non-damage


#Label and Split Data

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4436 to 4841
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1000 non-null   object
 1   text     1000 non-null   object
 2   label    1000 non-null   object
 3   label_2  1000 non-null   object
 4   label_3  1000 non-null   object
dtypes: object(5)
memory usage: 46.9+ KB


In [39]:
df.head()

,image,text,label,label_2,label_3
4436,"[[[[0.48235294 0.6627451 0.90588235], [0.4823...",It's always the moment to install solar contro...,building,non-damage,non-damage
5297,"[[[[1. 1. 1.], [1. 1. 1.], [1. 1. 1.], [1. 1. ...",Esmond lake \n@motobudz91 \n#lake #log #oregon...,landslide,landslide,damage
1121,"[[[[0.6745098 0.70980392 0.83529412], [0.6745...",Goodbye suit corner. #buildingcollapse #accide...,buildingcollapse,earthquake,damage
561,"[[[[0.88627451 0.8745098 0.83921569], [0.5725...",#somalia is a country that the world does not ...,terrorattack,war,damage
1188,"[[[[0.71372549 0.70196078 0.76862745], [0.7137...",NJ State Troopers patrolling The Streets of Un...,sandydamage,hurricane,damage


In [40]:
X_image = df['image']
X_text = df['text']
y = df['label_3']

In [41]:
new_X_image = []
for row in X_image:
    for image in row:
        new_X_image.append(image)

new_X_image[0]

X_image = [np.array(img).astype(np.float32) for img in new_X_image]

In [42]:
resized_X_image = []

for img in X_image:
    if img.shape != (250, 250, 3):

        resized_img = np.resize(img, (250, 250, 3))

        resized_X_image.append(resized_img)
    else:

        resized_X_image.append(img)

X_image = np.array(resized_X_image)

In [43]:
print(X_image.shape)
print(X_image[0].shape)

(1000, 250, 250, 3)
(250, 250, 3)


In [44]:
y = y.replace({'damage': 1, 'non-damage': 0})

<ipython-input-44-29e6a47d55c9>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'damage': 1, 'non-damage': 0})


In [45]:
X_train_image, X_test_image, X_train_text, X_test_text, y_train, y_test = train_test_split(X_image, X_text, y, test_size=0.2, random_state=42)


# Models

In [46]:
from tensorflow.keras.models import Sequential
# from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Add fully connected layers
model.add(Dense(64, activation='relu')) #changed leaky relu to relu
model.add(Dropout(0.5))  # Optional dropout for regularization

# model.add(Dense(64, activation='leaky_relu'))
# model.add(Dense(32, activation='leaky_relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 248, 248, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 124, 124, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 122, 122, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 238144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │      15,241,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,260,737 (58.22 MB)

 Trainable params: 15,260,737 (58.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=2) # Convert y_train to one-hot encoded format
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=2) # Convert y_test as well for consistency


In [ ]:
history = model.fit(X_train_image,
                    y_train,
                    epochs=10,
                    batch_size=32
                    )

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.5378 - loss: 1.5623
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.6554 - loss: 0.6502
Epoch 3/10
12/25 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.7555 - loss: 0.5362

In [33]:
model.evaluate(X_test_image, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5405 - loss: 0.8190


[0.7793723940849304, 0.5550000071525574]

In [35]:
predictions = model.predict(X_train_image)

25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step


In [36]:
def create_cnn_model():
    base_model = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(250, 250, 3))# using resnet 101 instead of 50
    base_model.trainable = False  # Freeze the pre-trained layers

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  # Reduce the spatial dimensions
        layers.Dense(100, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    return model

cnn_model = create_cnn_model()

# # Extract features from images
# image_features = cnn_model.predict(images)

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.evaluate(X_test_image, y_test)
predictions = cnn_model.predict(X_train_image)

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 125s 16s/step - accuracy: 0.4372 - loss: 0.7465
25/25 ━━━━━━━━━━━━━━━━━━━━ 472s 19s/step


In [ ]:
cnn_model.fit(X_train_image, y_train, epochs=10, batch_size=32)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 303s 12s/step - accuracy: 0.5063 - loss: 0.7060
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 320s 12s/step - accuracy: 0.4986 - loss: 0.7012
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 326s 12s/step - accuracy: 0.5435 - loss: 0.6891
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 296s 12s/step - accuracy: 0.5475 - loss: 0.6852
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 295s 12s/step - accuracy: 0.5275 - loss: 0.6866
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 323s 12s/step - accuracy: 0.5228 - loss: 0.6882
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 317s 12s/step - accuracy: 0.5461 - loss: 0.6817
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 319s 12s/step - accuracy: 0.5254 - loss: 0.6879
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 324s 12s/step - accuracy: 0.5120 - loss: 0.6999
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 288s 11s/step - accuracy: 0.5303 - loss: 0.6899


In [1]:
!pip install transformers

import tensorflow as tf
from transformers import GPT2Tokenizer, GPT2Model



 #Build the complete model

# CNN for image feature extraction
def create_cnn_model(input_shape):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the pre-trained layers

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Reduce dimensions
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    return base_model.input, x

image_input_shape = (250, 250, 3)
image_input, image_features = create_cnn_model(image_input_shape)

# # LSTM for text processing
# def create_lstm_model(vocab_size, embedding_dim, sequence_length):
#     text_input = tf.keras.Input(shape=(sequence_length,))
#     x = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=sequence_length)(text_input)
#     x = tf.keras.layers.LSTM(128)(x)
#     return text_input, x
# # Ensure tokenizer.word_index and max_sequence_length are defined before this point
# vocab_size = len(tokenizer.word_index) + 1
# embedding_dim = 128
# sequence_length = 100 #Replace with your actual sequence length

# text_input, text_features = create_lstm_model(vocab_size, embedding_dim, sequence_length)

# # Combine image features and text features
# combined_features = tf.keras.layers.Concatenate()([image_features, text_features])
# combined_output = tf.keras.layers.Dense(128, activation='relu')(combined_features)
# final_output = tf.keras.layers.Dense(1, activation='sigmoid')(combined_output)  # Binary classification

# # Build the complete model
# model = tf.keras.Model(inputs=[image_input, text_input], outputs=final_output)

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# # Model summary
# model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


NameError: name 'tokenizer' is not defined

In [ ]:
# Assuming `images` is a numpy array of image data
# Assuming `texts` is a list of raw text corresponding to the images

# Tokenize and pad the text data
# tokenizer = Tokenizer(num_words=10000)  # Adjust vocabulary size as needed
# tokenizer.fit_on_texts(texts)
# sequences = tokenizer.texts_to_sequences(texts)
# max_sequence_length = 100  # Define max sequence length
# padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# OR

# Using GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def tokenize_texts(texts):
    return tokenizer(texts, return_tensors='tf', padding=True, truncation=True, clean_up_tokenization_spaces=Fa

# Tokenize the text data
tokenized_texts = tokenize_texts(texts)

# Example disaster reports (this could be loaded from your dataset)
sample_reports = ["Flood in Texas causes widespread damage.",
                  "Earthquake in Japan leads to massive destruction."]
tokenized_reports = tokenize_texts(sample_reports)

# Assuming `labels` is a numpy array containing the target labels for the task (e.g., 0 and 1 for binary classification)


NameError: name 'texts' is not defined

In [ ]:
# Split the data into training and test sets
image_train, image_test, text_train, text_test, y_train, y_test = train_test_split(
    images, padded_sequences, labels, test_size=0.2, random_state=42)

print(f"Training set: {image_train.shape}, {text_train.shape}, {y_train.shape}")
print(f"Test set: {image_test.shape}, {text_test.shape}, {y_test.shape}")

In [ ]:
def create_lstm_model(image_feature_size, vocab_size, seq_length):
    image_input = layers.Input(shape=(image_feature_size,))
    text_input = layers.Input(shape=(seq_length,))

    # Combine Image and Text features
    x = layers.Concatenate()([image_input, text_input])
    x = layers.Embedding(vocab_size, 256)(x)
    x = layers.LSTM(256, return_sequences=True)(x)
    x = layers.Dense(vocab_size, activation='softmax')(x)

    model = models.Model(inputs=[image_input, text_input], outputs=x)
    return model

vocab_size = len(tokenizer)  # GPT tokenizer vocab size
seq_length = tokenized_reports['input_ids'].shape[1]  # Max sequence length
lstm_model = create_lstm_model(256, vocab_size, seq_length)

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
gpt2_model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Fine-tune GPT-2 model
def fine_tune_gpt2(image_features, texts, tokenizer):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True)
    outputs = gpt2_model(inputs['input_ids'], labels=inputs['input_ids'])
    loss = outputs.loss
    logits = outputs.logits

    # Train your model on this loss
    return loss

# Example training loop
for epoch in range(10):
    loss = fine_tune_gpt2(image_features, sample_reports, tokenizer)
    print(f"Epoch {epoch+1}, Loss: {loss}")


In [ ]:
# Save the CNN + LSTM model or GPT-2 based model
lstm_model.save("disaster_report_model.h5")

# For GPT-2 (Hugging Face)
gpt2_model.save_pretrained("disaster_gpt2_model")
tokenizer.save_pretrained("disaster_gpt2_model_tokenizer")


# Creating Web Application

In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from PIL import Image
import numpy as np

# Load the saved LSTM or GPT model
lstm_model = tf.keras.models.load_model('disaster_report_model.h5')

# If using GPT-2
gpt2_model = TFGPT2LMHeadModel.from_pretrained('disaster_gpt2_model')
tokenizer = GPT2Tokenizer.from_pretrained('disaster_gpt2_model_tokenizer')

app = Flask(__name__)

# Image preprocessing function
def preprocess_image(image, target_size=(224, 224)):
    image = Image.open(image)
    image = image.resize(target_size)
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Report generation endpoint
@app.route('/generate_report', methods=['POST'])
def generate_report():
    file = request.files['image']

    # Preprocess the input image
    image = preprocess_image(file)

    # Extract features from the image using CNN
    image_features = lstm_model.predict(image)  # You may want to extract features only

    # Use the LSTM or GPT model to generate a report
    # If LSTM:
    # generated_report = lstm_model.predict(image_features)

    # If GPT-2:
    input_text = tokenizer.decode(image_features[0])  # Convert image features to text for GPT-2
    input_ids = tokenizer.encode(input_text, return_tensors="tf")
    generated_report = gpt2_model.generate(input_ids, max_length=100, num_return_sequences=1)

    report = tokenizer.decode(generated_report[0], skip_special_tokens=True)

    return jsonify({"report": report})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
python app.py


In [ ]:
curl -X POST -F "image=@/path/to/disaster_image.jpg" http://127.0.0.1:5000/generate_report


In [ ]:
pip freeze > requirements.txt


In [ ]:
web: python app.py


In [ ]:
git init
heroku create
git add .
git commit -m "First commit"
git push heroku master
